# Negative Weighted Events

In this notebook, we adapt the negative events-based measure

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from ocpa.objects.log.importer.ocel import factory as ocel_import_factory
from ocpa.algo.discovery.ocpn import algorithm as ocpn_discovery_factory
from src.utils import get_happy_path_log, create_flower_model, generate_variant_model
from ocpa.objects.log.importer.csv import factory as ocel_import_factory_csv
from models.negative_events_measure_without_weighting import negative_events_without_weighting
import pickle

# O2C Log

### Standard Petri Net

In a first step, we load the OCEL-log into the notebook and generate the object-centric petri net.

In [3]:
filename = "../src/data/jsonocel/order_process.jsonocel"
ocel = ocel_import_factory.apply(filename)
ocpn = ocpn_discovery_factory.apply(ocel, parameters={"debug": False})

In [4]:
value = negative_events_without_weighting (ocel, ocpn)
value

Check the arcs: 100%|██████████| 46/46 [00:00<00:00, 44579.02it/s]
Calculate Generalization for all process executions: 100%|██████████| 48/48 [00:00<00:00, 54.57it/s]


0.5063

### Happy Path Petri Net

In [5]:
happy_path__ocel = get_happy_path_log(filename)

In [6]:
happy_path_ocpn = ocpn_discovery_factory.apply(happy_path__ocel, parameters={"debug": False})

In [7]:
value = negative_events_without_weighting (ocel, happy_path_ocpn)
value

Check the arcs: 100%|██████████| 26/26 [00:00<00:00, 26001.88it/s]
Calculate Generalization for all process executions: 100%|██████████| 48/48 [00:00<00:00, 78.48it/s]


0.3073

### Flower Model Petri Net

In [8]:
ots = ["order","item","delivery"]

In [9]:
flower_ocpn = create_flower_model(filename,ots)

In [10]:
value = negative_events_without_weighting (ocel, flower_ocpn)
value

Check the arcs: 100%|██████████| 32/32 [00:00<00:00, 37882.51it/s]
Calculate Generalization for all process executions: 100%|██████████| 48/48 [00:00<00:00, 77.92it/s]


0.965

### Variant Model Petri Net

Import the primarly generated variant log for our measure computation, while we generate the variant model with the original log.

In [11]:
filename_variant = "../src/data/csv/order_process_variant_log.csv" 
object_types = ["order","item","delivery"]
parameters = {"obj_names": object_types,
              "val_names": [],
              "act_name": "event_activity",
              "time_name": "event_timestamp",
              "sep": ","}
ocel_variant = ocel_import_factory_csv.apply(file_path=filename_variant, parameters=parameters)

In [12]:
filename = "../src/data/jsonocel/order_process.jsonocel"
ots = ["order","item","delivery"]
ocel = ocel_import_factory.apply(filename)
variant_ocpn = generate_variant_model(ocel,save_path_logs='../src/data/csv/order_variants/order_variant',object_types = ots,save_path_visuals=f"../reports/figures/order_variant_total.svg" )

Processing Variant Nets: 100%|██████████| 12/12 [00:00<00:00, 5984.74it/s]

#########Start generating Object-Centric Petri Net#########
#########Finished generating Object-Centric Petri Net#########


In [13]:
value = negative_events_without_weighting (ocel, variant_ocpn)
value

Check the arcs: 100%|██████████| 378/378 [00:00<00:00, 7777.33it/s]
Calculate Generalization for all process executions: 100%|██████████| 48/48 [00:00<00:00, 58.76it/s]


0.5652

# P2P Log

### Standard Petri Net

In a first step, we load the OCEL-log into the notebook and generate the object-centric petri net.

In [14]:
filename = "../src/data/jsonocel/p2p-normal.jsonocel"
ocel = ocel_import_factory.apply(filename)
ocpn = ocpn_discovery_factory.apply(ocel, parameters={"debug": False})

In [15]:
value = negative_events_without_weighting (ocel, ocpn)
value

Check the arcs: 100%|██████████| 40/40 [00:00<?, ?it/s]
Calculate Generalization for all process executions: 100%|██████████| 80/80 [00:00<00:00, 240.30it/s]


0.1845

### Happy Path Petri Net

In [16]:
happy_path__ocel = get_happy_path_log(filename)

In [17]:
happy_path_ocpn = ocpn_discovery_factory.apply(happy_path__ocel, parameters={"debug": False})

In [18]:
value = negative_events_without_weighting (ocel, happy_path_ocpn)
value

Check the arcs: 100%|██████████| 38/38 [00:00<?, ?it/s]
Calculate Generalization for all process executions: 100%|██████████| 80/80 [00:00<00:00, 245.38it/s]


0.1958

### Flower Model Petri Net

In [19]:
ots = ["PURCHORD","INVOICE","PURCHREQ","MATERIAL","GDSRCPT"]

In [20]:
flower_ocpn = create_flower_model(filename,ots)

In [21]:
value = negative_events_without_weighting (ocel, flower_ocpn)
value

Check the arcs: 100%|██████████| 38/38 [00:00<00:00, 37966.54it/s]
Calculate Generalization for all process executions: 100%|██████████| 80/80 [00:00<00:00, 289.83it/s]


0.8611

### Variant Model Petri Net

Import the primarly generated variant log for our measure computation, while we generate the variant model with the original log.

In [22]:
filename_variant = "../src/data/csv/p2p_variant_log.csv" 
object_types = ["PURCHORD","INVOICE","PURCHREQ","MATERIAL","GDSRCPT"]
parameters = {"obj_names": object_types,
              "val_names": [],
              "act_name": "event_activity",
              "time_name": "event_timestamp",
              "sep": ","}
ocel_variant = ocel_import_factory_csv.apply(file_path=filename_variant, parameters=parameters)

In [23]:
filename = "../src/data/jsonocel/p2p-normal.jsonocel"
ots = ["PURCHORD","INVOICE","PURCHREQ","MATERIAL","GDSRCPT"]
ocel = ocel_import_factory.apply(filename)
variant_ocpn = generate_variant_model(ocel,save_path_logs='../src/data/csv/p2p_variants/p2p_variant',object_types = ots ,save_path_visuals=f"../reports/figures/p2p_variant_total.svg" )

Processing Variant Nets: 100%|██████████| 20/20 [00:00<00:00, 3998.19it/s]

#########Start generating Object-Centric Petri Net#########
#########Finished generating Object-Centric Petri Net#########


In [24]:
value = negative_events_without_weighting (ocel, variant_ocpn)
value

Check the arcs: 100%|██████████| 760/760 [00:00<00:00, 10928.88it/s]
Calculate Generalization for all process executions: 100%|██████████| 80/80 [00:00<00:00, 249.61it/s]


0.7143

# BPI-Challenge 2017 Log

### Standard Petri Net

In a first step, we load the OCEL-log into the notebook and generate the object-centric petri net.

In [25]:
filename = "../src/data/jsonocel/BPI2017-Final.jsonocel"
ocel = ocel_import_factory.apply(filename)
ocpn = ocpn_discovery_factory.apply(ocel, parameters={"debug": False})

In [26]:
value = negative_events_without_weighting (ocel, ocpn)
value

Check the arcs: 100%|██████████| 120/120 [00:00<00:00, 119980.09it/s]
Calculate Generalization for all process executions: 100%|██████████| 31509/31509 [03:24<00:00, 154.40it/s]


0.3569

### Happy Path Petri Net

In [27]:
happy_path__ocel = get_happy_path_log(filename)

In [28]:
happy_path_ocpn = ocpn_discovery_factory.apply(happy_path__ocel, parameters={"debug": False})

In [29]:
value = negative_events_without_weighting (ocel, happy_path_ocpn)
value

Check the arcs: 100%|██████████| 26/26 [00:00<00:00, 76153.56it/s]
Calculate Generalization for all process executions: 100%|██████████| 31509/31509 [03:09<00:00, 166.08it/s]


0.1077

### Flower Model Petri Net

In [30]:
ots = ["application","offer"]

In [31]:
flower_ocpn = create_flower_model(filename,ots)

In [ ]:
value = negative_events_without_weighting (ocel, flower_ocpn)
value

Check the arcs: 100%|██████████| 56/56 [00:00<00:00, 56097.69it/s]
Calculate Generalization for all process executions:  52%|█████▏    | 16237/31509 [02:04<01:24, 181.50it/s]

### Variant Model Petri Net

Import the primarly generated variant log for our measure computation, while we generate the variant model with the original log.

In [ ]:
filename_variant = "../src/data/csv/bpi2017_variant_log.csv" 
object_types = ["application","offer"]
parameters = {"obj_names": object_types,
              "val_names": [],
              "act_name": "event_activity",
              "time_name": "event_timestamp",
              "sep": ","}
ocel_variant = ocel_import_factory_csv.apply(file_path=filename_variant, parameters=parameters)

We import the pickle file for the variant model of the bpi challenge that we generated in the process models notebook.

In [ ]:
with open("../src/data/csv/bpi_variant_ocpn.pickle", "rb") as file:
    variant_ocpn = pickle.load(file)

print(variant_ocpn)

In [ ]:
value = negative_events_without_weighting (ocel, variant_ocpn)
value